# Convolution Neural Network


**Convolution**

* [코넬대학교 CS1114 Section 6: Convolution](http://www.cs.cornell.edu/courses/cs1114/2013sp/sections/S06_convolution.pdf)
* [카네기멜론 Convolution and Edge Detection](http://graphics.cs.cmu.edu/courses/15-463/2005_fall/www/Lectures/convolution.pdf)
* [카네기멜론 On the Origin of Deep Learning](https://arxiv.org/pdf/1702.07800.pdf)
* [유타 Spatial Filtering](http://www.coe.utah.edu/~cs4640/slides/Lecture5.pdf)
* [The Convolution Operation and CNNs](http://www.cedar.buffalo.edu/~srihari/CSE676/ConvolutionFunction.pdf)
* [Introduction to Differential Equations](https://www.math.ust.hk/~machas/differential-equations.pdf)

## Convolution Operation

Convolution은 2개의 함수들 (e.g. $ f $ 그리고 $ g $)에 적용되는 일종의 덧셈 뺄셈과 같은 operations이며, 새로운 함수인 $ h $를 생성을 합니다. <br>
새로운 함수 $ h $는 함수 $ f $가 다른 함수 $ g $위를 지나가면서 중첩된 부분을 element-wise multiplication으로 연산뒤 각 구간별 integral을 나타냅니다.

$$ (f * g)(t) = \int^\infty_{-\infty} f(\mathcal{T}) g(t-\mathcal{T}) \ d\mathcal{T} $$


$$ h(t) (f * g)(t) =  $$

아래 그림에서, 왼쪽은 input matrix, 중간은 kernel matrix, 그리고 오른쪽은 결과물입니다. <br>
아래와 같이 element-wise product를 한뒤, sum을 해줍니다. 
<img src="./images/simple_convolution_operation.png">